# Graphs for the aero mod report

In [ ]:
import libs
import numpy as np
import pandas as pd
import plotly.graph_objects as go

matlab_template = dict(
        layout=go.Layout(
                #plot layout
                width=1300,
                height=800,
                showlegend=True,
                plot_bgcolor="white",
                margin=dict(t=200), #leave some padding on top

                #title
                title_font=dict(size=24, color="black", weight='bold',family="Arial",),
                title_x=0.5,  # Center the title horizontally
                title_y=0.98,  # Position the title slightly lower vertically

                #axis
                xaxis = dict(linecolor='black', showgrid=True, showline=True, gridcolor='lightgrey', ticks='outside',
                             tickcolor='black', tickwidth=2, ticklen=5,
                             title_font=dict(size=18, weight='bold', family='Arial'),
                             tickfont=dict(size=14, family='Arial'),
                             ),
                yaxis = dict(linecolor='black', showgrid=True, showline=True, gridcolor='lightgrey', ticks='outside',
                             tickcolor='black', tickwidth=2, ticklen=5,
                             title_font=dict(size=18, weight='bold', family='Arial'),
                             tickfont=dict(size=14, family='Arial'),
                             ),
                yaxis2 = dict(linecolor='black', showgrid=False, showline=True, gridcolor='lightgrey', ticks='outside',
                             tickcolor='black', tickwidth=2, ticklen=5,
                             title_font=dict(size=18, weight='bold', family='Arial'),
                             tickfont=dict(size=14, family='Arial'),
                             ),

                #legend
                legend=dict(
                        x=1.1,                # Position the legend at the far-right of the plot
                        y=1.1,                # Position the legend at the top of the plot
                        xanchor='right',    # Anchor the legend on the right side
                        yanchor='bottom',      # Anchor the legend at the top
                        bordercolor='black',# Border color around the legend (optional)
                        borderwidth=1       # Border width around the legend (optional)
                        ),
         )
)

## Lift curve

In [2]:
#Data for lift curve
data=pd.read_csv(r"data\Aeromod\2025-04-30_lift_coeff.csv")
ID=[] #ID of maneuvers to plot (if empty will plot all maneuvers).

In [ ]:
if ID==[]:
    #if no ID is specified, plot all maneuvers
    ID=data['ID'].unique()

selected_data=data[data['ID'].isin(ID)]


for airspeed_cat in ['120','150','180']:
    filtered_data = selected_data[selected_data['Label'].str.contains(airspeed_cat, na=False)]
    for conf, group in filtered_data.groupby('Conf'):

        #get theorical curves for CL
        CT=filtered_data['CT'].mean() #CT value for the maneuver. Used to draw the theorical CL curve.
        AOA=np.linspace(-2,19,100) #AOA range for the theorical CL curve.
        CL_vectorized=np.vectorize(libs.das.C12_CL_theo)
        CL_theo=CL_vectorized(CT,AOA,conf) #theorical CL curve.
        CL_theo__max=CL_theo+0.1
        CL_theo__min=CL_theo-0.1

        #get theorical curves for CD
        CD_vectorized=np.vectorize(libs.das.C12_CD_theo)
        CD_theo=CD_vectorized(CL_theo,CT,conf) #theorical CD curve.
        CD_theo__max=CD_theo+0.03
        CD_theo__min=CD_theo-0.03

        fig=go.Figure()
        fig2=go.Figure()

        fig.update_layout(
            template=matlab_template,
            # title=f"C-12C lift curve {maneuver_name} ({conf} configuration)",
            title="C-12C Lift Curve",
            xaxis=dict(title='Angle of attack, α (deg)',range=[-2,15],),
            yaxis=dict(title='Lif coefficient, C<sub>L</sub>',range=[0,1.5],),
        )
        #plot theorical curves for CL=f(alpha)
        fig.add_trace(go.Scatter(x=AOA, y=CL_theo,yaxis='y',mode='lines', name='theoretical CL curve',showlegend=False,line=dict(color='black')))
        fig.add_trace(go.Scatter(x=AOA, y=CL_theo__max,yaxis='y',mode='lines', name='theoretical CL curve +/- 0.1',showlegend=False,line=dict(color='black',dash='dash')))
        fig.add_trace(go.Scatter(x=AOA, y=CL_theo__min,yaxis='y',mode='lines', name='theoretical CL curve +/- 0.1',showlegend=False,line=dict(color='black',dash='dash')))

        fig2.update_layout(
            template=matlab_template,
            # title=f"C-12C drag polar {maneuver_name} ({conf} configuration)",
            title="C-12C Drag Polar",
            xaxis=dict(title='Drag coefficient, C<sub>D</sub>',),
            yaxis=dict(title='Lif coefficient, C<sub>L</sub>',),
        )
        #plot theorical curves for CL=f(CD)
        fig2.add_trace(go.Scatter(x=CD_theo, y=CL_theo,yaxis='y',mode='lines', name='theoretical CL curve',showlegend=False,line=dict(color='black')))
        fig2.add_trace(go.Scatter(x=CD_theo__max, y=CL_theo,yaxis='y',mode='lines', name='theoretical CL curve +/- 0.03',showlegend=False,line=dict(color='black',dash='dash')))
        fig2.add_trace(go.Scatter(x=CD_theo__min, y=CL_theo,yaxis='y',mode='lines', name='theoretical CL curve  +/- 0.03',showlegend=False,line=dict(color='black',dash='dash')))

        legend_RC=False
        legendW_UT=False
        #plot data by maneuver
        for conf_group in group.groupby('ID'):
            marker='cross' if 'RC' in conf_group['Label'].iloc[0] else 'triangle-up-open'
            maneuver='Roller coaster' if 'RC' in conf_group['Label'].iloc[0] else 'Wind-up turn'
            size=6 if 'RC' in conf_group['Label'].iloc[0] else 10
            color='grey' if 'RC' in conf_group['Label'].iloc[0] else 'black'

            if maneuver=='Roller coaster'and not legend_RC :
                showlegend=True
                legend_RC=True
            elif maneuver=='Wind-up turn' and not legendW_UT:
                showlegend=True
                legendW_UT=True
            else:
                showlegend=False
            fig.add_trace(go.Scatter(x=conf_group['AOA_true'], y=conf_group['CL'],yaxis='y',mode='markers', showlegend=showlegend, marker=dict(symbol=marker, size=size,color=color), name=maneuver))
            fig2.add_trace(go.Scatter(x=conf_group['CD'], y=conf_group['CL'],yaxis='y',mode='markers', showlegend=showlegend, marker=dict(symbol=marker,size=size,color=color), name=maneuver))


            fig.update_layout(width=8.42*180, height=6.4*180, legend=dict(yanchor="bottom", y=0.01, xanchor="right", x=0.99))
            fig2.update_layout(width=8.42*180, height=6.4*180, legend=dict(yanchor="bottom", y=0.01, xanchor="right", x=0.99))

            if conf=='Cruise':
                conf_extended='Cruise'
            elif conf=='Approach':
                conf_extended='Approach (gear down, flaps 40%)'
            elif conf=='Landing':
                conf_extended='Landing (gear down, flaps 40%)'

            #adding annotation on lift curve
            fig.add_shape(type="rect", xref="paper", yref="paper",
                x0=0, y0=1.05,     # Bottom-left corner
                x1=1, y1=1.15,     # Top-right corner
                line=dict(color="black", width=2,),
                layer="below",     # 'above' or 'below' the data
            )


            fig.add_annotation(x=0, y=1.15, xref="paper", yref="paper",align='left',
                font=dict(size=16, color="black",family="Arial"),
                text=f"<b>Configuration:</b> {conf_extended}<br><b>Pressure altitude:</b> 15 000 ft<br><b>Weight:</b> 11'500 lb to 12'000 lb<br><b>CG:</b> 20 to 20.5%<br>",
                showarrow=False,)

            fig.add_annotation(x=0.55, y=1.15, xref="paper", yref="paper",align='left',
                font=dict(size=16, color="black",family="Arial"),
                text=f"<b>Data basis:</b> Flight test<br><b>Test date:</b> April 30th, 2025<br><b>Tail number:</b> 215 (with YAPS)<br><b>Airspeed:</b> {airspeed_cat} KIAS<br>",
                showarrow=False,)

             #adding annotation on drag polar
            fig2.add_shape(type="rect", xref="paper", yref="paper",
                x0=0, y0=1.05,     # Bottom-left corner
                x1=1, y1=1.15,     # Top-right corner
                line=dict(color="black", width=2,),
                layer="below",     # 'above' or 'below' the data
            )

            fig2.add_annotation(x=0, y=1.15, xref="paper", yref="paper",align='left',
                font=dict(size=16, color="black",family="Arial"),
                text=f"<b>Configuration:</b> {conf_extended}<br><b>Pressure altitude:</b> 15 000 ft<br><b>Weight:</b> 11'500 lb to 12'000 lb<br><b>CG:</b> 20 to 20.5%<br>",
                showarrow=False,)

            fig2.add_annotation(x=0.55, y=1.15, xref="paper", yref="paper",align='left',
                font=dict(size=16, color="black",family="Arial"),
                text=f"<b>Data basis:</b> Flight test<br><b>Test date:</b> April 30th, 2025<br><b>Tail number:</b> 215 (with YAPS)<br><b>Airspeed:</b> {airspeed_cat} KIAS<br>",
                showarrow=False,)

            fig.update_layout(showlegend=True, legend=dict(font=dict(size=16, color="black",family="Arial"),),)
            fig2.update_layout(showlegend=True, legend=dict(font=dict(size=16, color="black",family="Arial"),),)

        fig.show()
        fig2.show()

        CD_error=filtered_data['CD']-filtered_data['CD_theo']

        # fig4=go.Figure()
        # fig4.update_layout(template=matlab_template,)
        # symbols = np.where(filtered_data['Label'].str.contains('Flip', na=False), 'circle', 'x')
        # fig4.add_trace(go.Scatter(x=filtered_data['AOSS'], y=CD_error,yaxis='y',mode='markers', marker=dict(symbol= symbols), name='CD error'))
        # fig4.show()

    # fig.write_image("data/Aeromod/lift_coeff_"+conf+".png", engine='kaleido')
    # fig2.write_image("data/Aeromod/drag_coeff_"+conf+".png", engine='kaleido')


# for column in ['AOA_true','Torque','CL']:
#     fig3=go.Figure()
#     fig3.update_layout(
#         template=matlab_template,
#         xaxis=dict(title=column),
#         yaxis=dict(title='Cd-CdTheo',),
#     )

#     fig3.add_trace(go.Scatter(x=filtered_data[column], y=CD_error,yaxis='y',mode='markers', marker=dict(symbol='circle'), name='CD error'))
#     fig3.show()

# fig5=go.Figure()
# fig5.update_layout(template=matlab_template,)
# symbols = np.where(filtered_data['Label'].str.contains('Flip', na=False), 'circle', 'x')
# fig5.add_trace(go.Scatter(x=filtered_data['CAS'], y=np.abs(filtered_data['AOSS']),yaxis='y',mode='markers', marker=dict(symbol= symbols), name='CD error'))
# fig5.show()